In [1]:
import pandas as pd
import warnings
import numpy as np
import numpy.typing as npt
import aes
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

warnings.filterwarnings("ignore", category=SyntaxWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

io_data = pd.read_csv('data/dpa-aes/aes_data.txt', header=None)
io_data = io_data.apply(lambda x: x.str.replace(' ', '')) # https://stackoverflow.com/questions/40083266/replace-comma-with-dot-pandas
io_data['in'] = io_data[0].str[:32]
io_data['out'] = io_data[0].str[32:]
io_data = io_data.drop(columns=[0])

# https://stackoverflow.com/questions/40083266/replace-comma-with-dot-pandas
traces = pd.read_csv('data/dpa-aes/aes_samples.txt', sep='\s+', decimal=',', header=None)
traces_np = traces.to_numpy()
# print(io_data['out'].iloc[0])

In [2]:
def hamming_weight(n):
    count = 0
    while n:
        count += n & 1
        n >>= 1
    return count

hamming_weights = np.array([hamming_weight(i) for i in range(256)])

In [4]:
numtraces = traces.shape[0]
numsamples = traces.shape[1]

sbox = np.asarray([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16], dtype=np.uint8)

def xor_sbox(pt, keyguess):
	return sbox[pt ^ keyguess]

bestguess = [0] * 16

for bnum in range(1): # 16
	maxcpa = np.zeros(256)
	for kguess in range(256):
		print(f'subkey {bnum}, hyp {hex(kguess)}')

		sumnum = np.zeros(numsamples)
		sumden1 = np.zeros(numsamples)
		sumden2 = np.zeros(numsamples)

		hyp = np.zeros(numtraces)
		for tnum, trace in enumerate(traces):
			tmp = [int(io_data['in'][tnum][i:i + 2], 16) for i in range(0, 32, 2)]
			in_matrix = np.asarray(tmp, dtype=np.uint8)
			hyp[tnum] = hamming_weights[xor_sbox(in_matrix[bnum], kguess)]

		meanh = np.mean(hyp, dtype=np.float64)
		meant = np.mean(traces, axis=0)

		for tnum in range(numtraces):
			hdiff = (hyp[tnum] - meanh)
			tdiff = traces_np[tnum, :] - meant

			sumnum = sumnum + (hdiff * tdiff)
			sumden1 = sumden1 + hdiff * hdiff
			sumden2 = sumden2 + tdiff * tdiff

		maxcpa[kguess] = np.max(np.abs(sumnum / np.sqrt(sumden1 * sumden2)))

		print(f'{maxcpa[kguess]=}')

	bestguess[bnum] = np.argmax(maxcpa)

print('Best Key Guess:')
for b in bestguess:
	print(hex(b))

subkey 0, hyp 0x0
maxcpa[kguess]=0.0420113039426524
subkey 0, hyp 0x1
maxcpa[kguess]=0.03874844606606725
subkey 0, hyp 0x2
maxcpa[kguess]=0.03852915515004502
subkey 0, hyp 0x3
maxcpa[kguess]=0.05281510343466979
subkey 0, hyp 0x4
maxcpa[kguess]=0.05253186403152655
subkey 0, hyp 0x5
maxcpa[kguess]=0.045338741731790595
subkey 0, hyp 0x6
maxcpa[kguess]=0.03801248294687602
subkey 0, hyp 0x7
maxcpa[kguess]=0.05069000219922463
subkey 0, hyp 0x8


KeyboardInterrupt: 

In [3]:
import numpy as np
import pandas as pd

# Load traces and inputs
traces = pd.read_csv('data/dpa-aes/aes_samples.txt', sep='\s+', decimal=',', header=None).values
num_samples = traces.shape[1]
num_traces = traces.shape[0]

io_data = pd.read_csv('data/dpa-aes/aes_data.txt', header=None)
io_data = io_data.apply(lambda x: x.str.replace(' ', ''))
io_data['in'] = io_data[0].str[:32]
io_data['out'] = io_data[0].str[32:]
io_data = io_data.drop(columns=[0])
inputs = io_data['in'].values

# Define the Sbox and its inverse
sbox = np.asarray([0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67, 0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59, 0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7, 0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1, 0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05, 0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83, 0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29, 0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b, 0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa, 0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c, 0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc, 0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec, 0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19, 0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee, 0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49, 0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79, 0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4, 0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6, 0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70, 0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9, 0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e, 0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1, 0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0, 0x54, 0xbb, 0x16], dtype=np.uint8) # Load the AES Sbox # Load the AES Sbox

# Define the hypothetical power consumption model
def power_model(byte, value):
    """
    Compute the hypothetical power consumption for a given byte and value.
    """
    output = sbox[byte ^ value]
    hamming_weight = hamming_weights[output]
    return hamming_weight

# Perform CPA attack
for byte_index in range(16):
    print(f'box {byte_index}')
    max_correlation = 0
    best_key_guess = None

    for key_guess in range(256):
        hypothetical_values = []
        for trace_index in range(num_traces):
            input_bytes = np.array([int(inputs[trace_index][i:i+2], 16) for i in range(0, len(inputs[trace_index]), 2)])
            hypothetical_values.append(power_model(input_bytes[byte_index], key_guess))
        hypothetical_values = np.array(hypothetical_values)

        for sample_index in range(num_samples):
            correlation = np.corrcoef(hypothetical_values, traces[:, sample_index])[0, 1]
            if abs(correlation) > max_correlation:
                max_correlation = abs(correlation)
                best_key_guess = key_guess

    print(f"Recovered key byte {byte_index}: {best_key_guess:02x}")

box 0
Recovered key byte 0: ca
box 1


KeyboardInterrupt: 

In [38]:
all_hamming_weights_per_box = np.zeros((5000, 16), dtype=np.uint8)
coeff = np.zeros((256, 16))

for key_guess in range(256):
    simulated_traces = np.zeros((5000, 528))
    print(f'Analyzing key byte guess: {hex(key_guess)}')
    key_guess_matrix = np.full((4, 4), key_guess, dtype=np.uint8)
    for tnum, in_data in enumerate(io_data['in']):
        # fancy way to parse hex data and put it into a 4x4 matrix
        tmp = [int(in_data[i:i + 2], 16) for i in range(0, len(in_data), 2)]
        in_matrix = np.asarray(tmp, dtype=np.uint8).reshape(4, 4)

        added_key = aes.add_round_key(in_matrix, key_guess_matrix)
        sub_out = aes.sub_bytes(added_key, False).flatten()

        for bnum in range(16):
            all_hamming_weights_per_box[tnum][bnum] = hamming_weights[sub_out[bnum]]

    for bnum in range(16):
        simulated_traces_sum = np.sum((traces.T * all_hamming_weights_per_box[:, bnum]).T, axis=0)
        coeff[key_guess][bnum] = np.abs(pearsonr(np.sum(traces, axis=0), simulated_traces_sum)[0])

Analyzing key byte guess: 0x0
Analyzing key byte guess: 0x1
Analyzing key byte guess: 0x2
Analyzing key byte guess: 0x3
Analyzing key byte guess: 0x4
Analyzing key byte guess: 0x5
Analyzing key byte guess: 0x6
Analyzing key byte guess: 0x7
Analyzing key byte guess: 0x8
Analyzing key byte guess: 0x9
Analyzing key byte guess: 0xa
Analyzing key byte guess: 0xb
Analyzing key byte guess: 0xc
Analyzing key byte guess: 0xd
Analyzing key byte guess: 0xe
Analyzing key byte guess: 0xf
Analyzing key byte guess: 0x10
Analyzing key byte guess: 0x11
Analyzing key byte guess: 0x12
Analyzing key byte guess: 0x13
Analyzing key byte guess: 0x14
Analyzing key byte guess: 0x15
Analyzing key byte guess: 0x16
Analyzing key byte guess: 0x17
Analyzing key byte guess: 0x18
Analyzing key byte guess: 0x19
Analyzing key byte guess: 0x1a
Analyzing key byte guess: 0x1b
Analyzing key byte guess: 0x1c
Analyzing key byte guess: 0x1d
Analyzing key byte guess: 0x1e
Analyzing key byte guess: 0x1f
Analyzing key byte guess

In [25]:
np.sum((traces.T * all_hamming_weights_per_box[:, 0]).T, axis=0).shape

(528,)

In [39]:
np.set_printoptions(formatter={'int':hex})
np.argsort(coeff, axis=0)[0]

array([0xca, 0xfe, 0xba, 0xbe, 0xca, 0xfe, 0xba, 0xbe, 0xca, 0xfe, 0xba,
       0xbe, 0xca, 0xfe, 0xba, 0xbe])

In [5]:
all_hamming_weights_per_box3 = np.zeros((5000, 16), dtype=np.uint8)
coeff2 = np.zeros((16, 256))

for key_guess in range(1):
    print(f'Analyzing key byte guess: {hex(key_guess)}')
    key_guess_matrix = np.full((4, 4), key_guess, dtype=np.uint8)
    for tnum, in_data in enumerate(io_data['in']):
        # fancy way to parse hex data and put it into a 4x4 matrix
        tmp = [int(in_data[i:i + 2], 16) for i in range(0, len(in_data), 2)]
        in_matrix = np.asarray(tmp, dtype=np.uint8).reshape(4, 4)

        added_key = aes.add_round_key(in_matrix, key_guess_matrix)
        sub_out = aes.sub_bytes(added_key, False).flatten()

        for byte_index in range(sub_out.shape[0]):
            all_hamming_weights_per_box3[tnum][byte_index] = hamming_weights[sub_out[byte_index]]

    # for box in range(16):
    #     guess_box_weights = all_hamming_weights_per_box2[:, key_guess, box]
    #     WxH = (traces.T * guess_box_weights).T
    #     EWxH = np.sum(WxH, axis=0)

    #     Ew = np.sum(traces, axis=0)
    #     Eh = np.sum(guess_box_weights)

    #     Ew2 = np.sum(np.square(traces), axis=0)
    #     Ew_squared = np.square(Ew)

    #     Eh2 = np.sum(np.square(guess_box_weights), axis=0)
    #     Eh_squared = np.square(Eh)

    #     top = (5000 * EWxH) - (Eh * Ew)
    #     bottom = np.sqrt((5000 * Ew2) - Ew_squared) * np.sqrt((5000 * Eh2) - Eh_squared)
    #     coeff2[box][key_guess] = np.max(top / bottom)


Analyzing key byte guess: 0x0


In [3]:
all_hamming_weights_per_box2 = np.zeros((5000, 256, 16), dtype=np.uint8)
coeff2 = np.zeros((16, 256))

for key_guess in range(256):
    print(f'Analyzing key byte guess: {hex(key_guess)}')
    key_guess_matrix = np.full((4, 4), key_guess, dtype=np.uint8)
    for tnum, in_data in enumerate(io_data['in']):
        # fancy way to parse hex data and put it into a 4x4 matrix
        tmp = [int(in_data[i:i + 2], 16) for i in range(0, len(in_data), 2)]
        in_matrix = np.asarray(tmp, dtype=np.uint8).reshape(4, 4)

        added_key = aes.add_round_key(in_matrix, key_guess_matrix)
        sub_out = aes.sub_bytes(added_key, False).flatten()

        for byte_index in range(sub_out.shape[0]):
            all_hamming_weights_per_box2[tnum][key_guess][byte_index] = hamming_weights[sub_out[byte_index]]

    for box in range(16):
        guess_box_weights = all_hamming_weights_per_box2[:, key_guess, box]
        WxH = (traces.T * guess_box_weights).T
        EWxH = np.sum(WxH, axis=0)

        Ew = np.sum(traces, axis=0)
        Eh = np.sum(guess_box_weights)

        Ew2 = np.sum(np.square(traces), axis=0)
        Ew_squared = np.square(Ew)

        Eh2 = np.sum(np.square(guess_box_weights), axis=0)
        Eh_squared = np.square(Eh)

        top = (5000 * EWxH) - (Eh * Ew)
        bottom = np.sqrt((5000 * Ew2) - Ew_squared) * np.sqrt((5000 * Eh2) - Eh_squared)
        coeff2[box][key_guess] = np.max(top / bottom)


Analyzing key byte guess: 0x0
Analyzing key byte guess: 0x1
Analyzing key byte guess: 0x2
Analyzing key byte guess: 0x3
Analyzing key byte guess: 0x4
Analyzing key byte guess: 0x5
Analyzing key byte guess: 0x6
Analyzing key byte guess: 0x7
Analyzing key byte guess: 0x8
Analyzing key byte guess: 0x9
Analyzing key byte guess: 0xa
Analyzing key byte guess: 0xb
Analyzing key byte guess: 0xc
Analyzing key byte guess: 0xd
Analyzing key byte guess: 0xe
Analyzing key byte guess: 0xf
Analyzing key byte guess: 0x10
Analyzing key byte guess: 0x11
Analyzing key byte guess: 0x12
Analyzing key byte guess: 0x13
Analyzing key byte guess: 0x14
Analyzing key byte guess: 0x15
Analyzing key byte guess: 0x16
Analyzing key byte guess: 0x17
Analyzing key byte guess: 0x18
Analyzing key byte guess: 0x19
Analyzing key byte guess: 0x1a
Analyzing key byte guess: 0x1b
Analyzing key byte guess: 0x1c
Analyzing key byte guess: 0x1d
Analyzing key byte guess: 0x1e
Analyzing key byte guess: 0x1f
Analyzing key byte guess

In [10]:
# This can be done inside the main loop...
# for box in range(16):
#     for guess in range(2**8):
#         guess_box_weights = all_hamming_weights_per_box[:, guess, box]
#         WxH = (traces.T * guess_box_weights).T
#         EWxH = np.sum(WxH, axis=0)

#         Ew = np.sum(traces, axis=0)
#         Eh = np.sum(guess_box_weights)

#         Ew2 = np.sum(np.square(traces), axis=0)
#         Ew_squared = np.square(Ew)

#         Eh2 = np.sum(np.square(guess_box_weights), axis=0)
#         Eh_squared = np.square(Eh)

#         top = (5000 * EWxH) - (Eh * Ew)
#         bottom = np.sqrt((5000 * Ew2) - Ew_squared) * np.sqrt((5000 * Eh2) - Eh_squared)
#         coeff[box][guess] = np.max(top / bottom)

In [4]:
for box in range(16):
    print(hex(np.argsort(coeff2[box, :])[-1]))

0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe
0xca
0xfe
0xba
0xbe


In [23]:
coeff[0xcb, 0]

0.9299721307798332

In [11]:
coeff2.T[0xca, 0]

0.9455776059753018